## Library

In [2]:
import os
import statistics
import mysql.connector as MariaDB
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import join
from pathlib import Path

## DB Connection

In [ ]:
user = os.environ.get('USER')
pw = os.environ.get('PASSWORD')

In [4]:
con = MariaDB.connect(
    host="127.0.0.1",
    user='root',
    passwd='pw_root',
    db="DB_TEST",
    port=3307
)

## Statistics Table

### Getting data from round statistics table

In [ ]:
table = pd.read_sql("SELECT * FROM round_statistics", con)
table

### Auxiliary tables:

#### Total time of operator and main function per operators

In [ ]:
df1 = pd.DataFrame(data=table[["id", "operator_total_time", "operator"]].rename(columns={"operator_total_time":"time"}))
df1["obs"] = "tempo_total_operador"
df2 = pd.DataFrame(data=table[["id", "function_total_time", "operator"]].rename(columns={"function_total_time":"time"}))
df2["obs"] = "tempo_total_função"
df = pd.concat([df1, df2], ignore_index=True)
df

#### Time per operator

In [ ]:
time_operators = df.query("obs == 'tempo_total_operador'").groupby(["id","operator"]).sum("time").reset_index()
time_operators

#### Total time of operator and main function

In [ ]:
all_process = table.groupby("id").sum().rename(columns={"operator_total_time": "tempo_total_operador", "function_total_time": "tempo_total_função"})
all_process.head()

### Graphs

In [ ]:
path_imgs = join(
    str(Path(os.path.abspath('')).parents[0]),
    "imgs",
    "{filename}"
    )
sns.set_style("whitegrid")

* Total time of operator and main function per operator

In [ ]:
plt.figure(figsize = (15,8))
grid = sns.FacetGrid(df.rename(columns={"operator": "Operadores"}), col="obs", hue="Operadores", despine=False, size=5, aspect=1)
grid.map(sns.regplot, "id", "time")
grid.set_axis_labels("Iteração", "Tempo (s)")
grid.set_titles(col_template="{col_name}")
grid.set(yticks=[0,0.5, 1,1.5,2,2.5,3,3.5,4,4.5], xticks=[0, 10, 20, 30, 40, 50])
grid.add_legend()
grid.fig.subplots_adjust(top=0.85)
grid.fig.suptitle('Tempo de processamento para cada iteração por operador', fontsize=16)
plt.show()
grid.savefig(path_imgs.format(filename="per_operator.png"), format="png", dpi=grid.fig.dpi)

* Total time for each iteraction

In [ ]:
fig, axes = plt.subplots(figsize=(10,5))
sns.lineplot(ax=axes, x="id", y="tempo_total_operador", data=all_process)
sns.lineplot(ax=axes, x="id", y="tempo_total_função", data=all_process)
axes.set_title("Tempo total para processamento em cada iteração", fontsize="17", y=1.05)
axes.set(ylabel="Tempo (s)", xlabel="Iteração")
plt.legend(labels=["Operadores","Funções"])
plt.show()
fig.savefig(path_imgs.format(filename="total_processing.png"), dpi=fig.dpi, format="png")

### Metrics

* Compound Iteration Growth Rate (CIRG) 

In [ ]:
max_interactions = time_operators["id"].max()
min_interactions = time_operators["id"].min()

final_values = time_operators.loc[
        ((df.id == max_interactions) & (df.operator == 'generator')) |
        ((df.id == max_interactions) & (df.operator == 'import'))
        ].iloc[:,:]
initial_values = time_operators.loc[
        ((df.id == min_interactions) & (df.operator == 'generator')) |
        ((df.id == min_interactions) & (df.operator == 'import'))
        ].iloc[:,:]

In [ ]:
final_value_o = final_values.loc[(final_values.operator == 'generator'), 'time'].values[0]
initial_value_o = initial_values.loc[(initial_values.operator == 'generator'), 'time'].values[0]

CIGR_O = ((final_value_o/initial_value_o)**(1/max_interactions)-1)*100
CIGR_O

In [ ]:
final_value_o = final_values.loc[(final_values.operator == 'import'), 'time'].values[0]
initial_value_o = initial_values.loc[(initial_values.operator == 'import'), 'time'].values[0]

CIGR_I = ((final_value_o/initial_value_o)**(1/max_interactions)-1)*100
CIGR_I

* Standard Deviation

In [ ]:
statistics.pstdev(time_operators.loc[(time_operators.operator == 'generator'), 'time'].values)

* Median

In [ ]:
diff_time = all_process['tempo_total_operador'] - all_process["tempo_total_função"]
statistics.median(diff_time)

* Total time

In [ ]:
time_operators.groupby('operator').sum('time').reset_index()

In [ ]:
time_operators['time'].sum()